## Setup libraries

In [15]:
from math import sqrt
import numpy as np
import pandas as pd
from pandasql import sqldf
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import EarlyStopping

# Setup tables

In [16]:
Moggio = read_csv("Moggiodata14_20.csv")
Moggio = Moggio.loc[:, ["Date", "Ammonia", "PM10", "PM25",
                        "Wind_speed", "Wind_direction", "Temperature", "Rainfall"]]
# Cancella su tutta la riga anche solo con un mancante
Moggio = Moggio.dropna(how="any")
Moggio = Moggio.reset_index()
Moggio = Moggio.iloc[:730, :]  # we train not using 2020, for now

Moggio.head()


,index,Date,Ammonia,PM10,PM25,Wind_speed,Wind_direction,Temperature,Rainfall
0,0,2014-01-01T00:00:00Z,0.0,6.0,7.0,0.443554,99.0,-0.334965,0.0
1,1,2014-01-02T00:00:00Z,0.0,6.0,5.0,0.355915,105.0,-0.486111,7.8
2,2,2014-01-03T00:00:00Z,0.0,11.0,4.0,0.207631,120.0,2.220588,0.0
3,3,2014-01-04T00:00:00Z,0.0,6.0,7.0,1.024926,175.0,2.137063,35.8
4,4,2014-01-05T00:00:00Z,0.0,3.0,8.0,0.666721,94.0,2.416084,25.0


In [17]:
#Primo quadrante
Q1 = Moggio.loc[((Moggio.Wind_direction > 0) &
                 (Moggio.Wind_direction < 90)), :]

a = Q1.shape[0]
arr1 = []
arr1 = [1 for i in range(a)]

del Q1['Wind_direction']
Q1["Wind_direction"] = arr1
columns_titles = ["index", "Date", "Ammonia", "PM10", "PM25",
                  "Wind_speed",  "Temperature", "Rainfall", "Wind_direction", ]
Q1 = Q1.reindex(columns=columns_titles)

#Secondo quadrante
Q2 = Moggio.loc[((Moggio.Wind_direction > 90) &
                 (Moggio.Wind_direction < 180)), :]
a = Q2.shape[0]
arr2 = []
arr2 = [2 for i in range(a)]

del Q2['Wind_direction']
Q2["Wind_direction"] = arr2

#Terzo quadrante
Q3 = Moggio.loc[((Moggio.Wind_direction > 180) &
                 (Moggio.Wind_direction < 270)), :]

a = Q3.shape[0]
arr3 = []
arr3 = [3 for i in range(a)]

del Q3['Wind_direction']
Q3["Wind_direction"] = arr3

#Quarto quadrante
Q4 = Moggio.loc[((Moggio.Wind_direction > 270) &
                 (Moggio.Wind_direction < 360)), :]

a = Q4.shape[0]
arr4 = []
arr4 = [4 for i in range(a)]

del Q4['Wind_direction']
Q4["Wind_direction"] = arr4

Moggio = sqldf('''
SELECT *
FROM Q1
UNION
SELECT *
FROM Q2
UNION
SELECT *
FROM Q3
UNION
SELECT *
FROM Q4
''')
#del Moggio['Wind_direction']
Moggio.rename(columns={'Wind_direction': 'Quadrante'}, inplace=True)

Moggio

/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_6113/1396710986.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q1["Wind_direction"] = arr1
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_6113/1396710986.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q2["Wind_direction"] = arr2
/var/folders/m5/1xdj8pfj2cs3wjpc41g9qkj80000gn/T/ipykernel_6113/1396710986.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,index,Date,Ammonia,PM10,PM25,Wind_speed,Temperature,Rainfall,Quadrante
0,0,2014-01-01T00:00:00Z,0.0,6.0,7.0,0.443554,-0.334965,0.0,2
1,1,2014-01-02T00:00:00Z,0.0,6.0,5.0,0.355915,-0.486111,7.8,2
2,2,2014-01-03T00:00:00Z,0.0,11.0,4.0,0.207631,2.220588,0.0,2
3,3,2014-01-04T00:00:00Z,0.0,6.0,7.0,1.024926,2.137063,35.8,2
4,4,2014-01-05T00:00:00Z,0.0,3.0,8.0,0.666721,2.416084,25.0,2
...,...,...,...,...,...,...,...,...,...
720,999,2016-09-26T00:00:00Z,1.8,25.0,20.0,0.743963,13.533333,0.0,2
721,1000,2016-09-27T00:00:00Z,2.3,36.0,30.0,0.712583,12.205556,20.0,2
722,1001,2016-09-28T00:00:00Z,4.1,40.0,31.0,0.112516,13.405556,0.0,1
723,1003,2016-09-30T00:00:00Z,2.8,40.0,22.0,0.365800,12.690278,0.0,2


In [19]:
Moggio.to_csv('/Users/marcovinciguerra/Github/GitTesi/Python/Quadranti.csv')

